In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
my_drive_path = '/content/drive/MyDrive'
sys.path.append(my_drive_path)

In [ ]:
pip install evaluate bleu sacrebleu

In [ ]:
from sklearn.model_selection import train_test_split

# BLEU evaluation:
import nltk
from nltk.translate.bleu_score import sentence_bleu

from datasets.dataset_dict import DatasetDict
from datasets import Dataset
import evaluate

import pandas as pd
import numpy as np
import warnings
import pickle

warnings.filterwarnings('ignore')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, pipeline
from transformers import NllbTokenizer
from transformers import AutoConfig

In [ ]:
# Load local functions:
from database_object import * # read the data and create the dataset
from functions import *
from functions_model import *

# DATA COLLECTION

In [ ]:
db = database() # Create a database object

# Read the data and fill the database:
db.get_data_alsaimmer(display=False)
db.get_data_alsatext(display=False)
db.get_data_motsAlsacienMulhouse(display=False)
db.get_data_alignments(display=False)


In [ ]:
# Split the dataset into training/validation/testing subsets :
train, validtest = train_test_split(db.db    , test_size=0.6, random_state=0) # 60% training
valid, test      = train_test_split(validtest, test_size=0.5, random_state=0) # 20%-20% valid, test

# Create the dataset in a 'Dataset' format for tokenization :
d = {'train'     : Dataset.from_dict({'translation': train}),
     'validation': Dataset.from_dict({'translation': valid}),
     'test'      : Dataset.from_dict({'translation': test})
     }
d = DatasetDict(d)

# PRE-PROCESSING

In [ ]:
# Choice of Tokenizer:
checkpoint = "google-t5/t5-base" # "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
config = AutoConfig.from_pretrained(checkpoint)

#checkpoint = "facebook/nllb-200-distilled-600M" # too much demanding for colab (RAM excedeed, even with batch_size=1)
#checkpoint = "facebook/mbart-large-50-many-to-many-mmt" # same
#tokenizer = NllbTokenizer.from_pretrained(checkpoint)

src_lang = "fr"
tgt_lang = "als" # target language set to german as Alsatian as it is not supported

# translate French to Alsatian:
tokenizer.src_lang = src_lang
tokenizer.tgt_lang = tgt_lang

# Print an example of sentence and their tokens:
with tokenizer.as_target_tokenizer():
  tokens_fr = tokenizer(db.db[0]['fr'])
print("---- FRENCH ------------------------")
print(db.db[0]['fr'])
print(tokens_fr)

with tokenizer.as_target_tokenizer():
  tokens_als = tokenizer(db.db[0]['als'])
print("---- ALSACIAN ----------------------")
print(db.db[0]['als'])
print(tokens_als)


In [ ]:
tokenized_datasets = d.map(encode, batched=True, fn_kwargs={"tokenizer":tokenizer})

# MODEL TUNING

In [ ]:
# Loading of the pre-trained model for fine-tuning with additional dataset:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
# Set the output directory for saving the model:
model_name = checkpoint.split("/")[-1] # folder name to save output checkpoints

output_dir = "%s/runs/%s_essai6_evaluation"%(my_drive_path, model_name)

In [ ]:
# Set the arguments for training :
args = Seq2SeqTrainingArguments(
    output_dir,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 2e-4,
    per_device_train_batch_size = 8, # 64 is too much for colab -> RAM overflow
    per_device_eval_batch_size  = 8,
    save_steps       = 512, # save every 512 steps.
    weight_decay     = 0.01,
    save_total_limit = 1,
    num_train_epochs = 20,
    predict_with_generate = True,
    fp16                  = True, # True to speed up training on GPU
    metric_for_best_model = 'eval_loss',
    greater_is_better     = False,
    load_best_model_at_end= True,
    seed=1,
  )

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
# Evaluation function
# (using references from https://github.com/huggingface/transformers/issues/24433 and https://github.com/huggingface/transformers/issues/22634

metric = evaluate.load("sacrebleu")

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    # HF may return a tuple; take first element
    if isinstance(preds, (tuple, list)):
      preds = preds[0]

    # If logits slipped in (B, T, V), convert to token ids safely
    if preds.ndim == 3:
      preds = np.argmax(preds, axis=-1)

    # Map ignore index to a real token id for decoding
    pad_id = tokenizer.pad_token_id
    preds  = np.where(preds != -100, preds, pad_id)
    labels = np.where(labels != -100, labels, pad_id)

    pred_seq  = tokenizer.batch_decode(preds,  skip_special_tokens=True)
    label_seq = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(pred_seq, label_seq)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    return result

# MODEL TRAINING

In [ ]:
# Set up the trainer:
trainer = Seq2SeqTrainer(model, args,
    train_dataset    = tokenized_datasets["train"],
    eval_dataset     = tokenized_datasets["validation"],
    data_collator    = data_collator,
    processing_class = tokenizer,
    compute_metrics  = compute_metrics,
)

In [ ]:
# Do the training:
try:
  trainer.train(resume_from_checkpoint=True)
except:
  trainer.train(resume_from_checkpoint=False)

# MODEL EVALUATION

In [ ]:
# After training, evaluate the model on test set :
trainer.evaluate()

In [ ]:
# Save the model for future use
trainer.save_model(output_dir)
config.save_pretrained(output_dir)

#tokenizer.save_pretrained(output_dir)

# save dataset into pickle:
with open('%s/tokenized_datasets.pckl'%output_dir, 'wb') as fic:
  pickle.dump(tokenized_datasets, fic)

In [ ]:
# --------------------------------------------------------------------------------------------------
def do_translation(text, model_name, return_text=True, return_token=False):
  """
  Do the translation using the trained model.
  return either the translation or the tokens
  """
  # make the text a correct format for translation :
  translator = pipeline("translation_XX_to_YY", model=model_name)
  translator(text)

  # Tokenize the text: text -> tokens :
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  inputs = tokenizer(text, return_tensors="pt").input_ids

  model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
  # Do the translation using the tokenized text:
  outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)

  # Token -> text:
  translation = tokenizer.decode(outputs[0], skip_special_tokens=True)

  if return_text:
    return translation
  if return_tokens:
    return output[0]

# TRY SOME TRANSLATION

In [ ]:
text = "Le château est sur la montagne."

translation = do_translation(text, output_dir)
print("%s -> %s"%(text, translation))